In [35]:
import pandas as pd
import numpy as np
import pdpipe as pdp

In [92]:
# Load data
df = pd.read_csv('data/USA_Housing.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Avg. Area Income              5000 non-null   float64
 1   Avg. Area House Age           5000 non-null   float64
 2   Avg. Area Number of Rooms     5000 non-null   float64
 3   Avg. Area Number of Bedrooms  5000 non-null   float64
 4   Area Population               5000 non-null   float64
 5   Price                         5000 non-null   float64
 6   Address                       5000 non-null   object 
dtypes: float64(6), object(1)
memory usage: 273.6+ KB


,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701..."
1,79248.642455,6.002900,6.730821,3.09,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA..."
2,61287.067179,5.865890,8.512727,5.13,36882.159400,1.058988e+06,"9127 Elizabeth Stravenue\nDanieltown, WI 06482..."
3,63345.240046,7.188236,5.586729,3.26,34310.242831,1.260617e+06,USS Barnett\nFPO AP 44820
4,59982.197226,5.040555,7.839388,4.23,26354.109472,6.309435e+05,USNS Raymond\nFPO AE 09386
...,...,...,...,...,...,...,...
4995,60567.944140,7.830362,6.137356,3.46,22837.361035,1.060194e+06,USNS Williams\nFPO AP 30153-7653
4996,78491.275435,6.999135,6.576763,4.02,25616.115489,1.482618e+06,"PSC 9258, Box 8489\nAPO AA 42991-3352"
4997,63390.686886,7.250591,4.805081,2.13,33266.145490,1.030730e+06,"4215 Tracy Garden Suite 076\nJoshualand, VA 01..."
4998,68001.331235,5.534388,7.130144,5.44,42625.620156,1.198657e+06,USS Wallace\nFPO AE 73316


In [19]:
df.isna().sum()

Avg. Area Income                0
Avg. Area House Age             0
Avg. Area Number of Rooms       0
Avg. Area Number of Bedrooms    0
Area Population                 0
Price                           0
Address                         0
dtype: int64

In [85]:
# insert NaN to dataset
np.random.seed(42)
df_miss = df.mask(np.random.random(df.shape) < .1)
df_miss.isna().sum()

Avg. Area Income                934
Avg. Area House Age             986
Avg. Area Number of Rooms       944
Avg. Area Number of Bedrooms    950
Area Population                 919
Price                           949
Address                         954
dtype: int64

In [86]:
# def home size function
def size(n):
    if n <= 4:
        return 'Small'
    elif 4 < n <= 6:
        return 'Medium'
    else:
        return 'Big'
df_miss['House_size'] = df_miss['Avg. Area Number of Rooms'].apply(size)

In [87]:
# 1. Pipe One-Hot-Encode
pipeline = pdp.OneHotEncode('House_size')

In [91]:
def price_tag(x):
    if x > 2:
        return 'keep'
    else:
        return 'drop'
# 2. Pipe Drop column 
pipeline += pdp.RowDrop({'Price': lambda x: x<= 250000})

# 3. Pipe Apply func by cols
pipeline += pdp.ApplyByCols('Price', price_tag, 'Price_tag', drop=False)
pipeline(df_miss)
# pipeline += pdp.ColDrop('Avg. Area House Age')

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,House_size_Medium,House_size_Small
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,NaN,0,0
1,79248.642455,6.002900,6.730821,NaN,NaN,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA...",0,0
2,61287.067179,5.865890,8.512727,5.13,NaN,1.058988e+06,NaN,0,0
3,63345.240046,7.188236,5.586729,3.26,34310.242831,1.260617e+06,USS Barnett\nFPO AP 44820,1,0
4,59982.197226,NaN,7.839388,4.23,NaN,6.309435e+05,USNS Raymond\nFPO AE 09386,0,0
...,...,...,...,...,...,...,...,...,...
4995,60567.944140,7.830362,6.137356,NaN,22837.361035,1.060194e+06,USNS Williams\nFPO AP 30153-7653,0,0
4996,78491.275435,6.999135,NaN,4.02,NaN,1.482618e+06,"PSC 9258, Box 8489\nAPO AA 42991-3352",0,0
4997,63390.686886,7.250591,4.805081,2.13,33266.145490,1.030730e+06,"4215 Tracy Garden Suite 076\nJoshualand, VA 01...",1,0
4998,68001.331235,5.534388,7.130144,5.44,42625.620156,NaN,USS Wallace\nFPO AE 73316,0,0


In [ ]:
df2 = pipeline(df_miss)